# Installing of some libraries(can be skip if you work not in Google Colab)

In [ ]:
!pip install gtts

In [ ]:
!pip install deep_translator

#Libraries

In [ ]:
from diffusers import AutoPipelineForText2Image
from time import sleep
import threading
import asyncio
from concurrent.futures import ThreadPoolExecutor
import google.generativeai as genai
from io import StringIO
from gtts import gTTS
from moviepy.editor import *
import librosa
import numpy as np
from scipy.io import wavfile
from scipy.signal import butter, filtfilt
import torch
from deep_translator import GoogleTranslator

# Gemini api key

In [ ]:
API_KEY = input("Get api-key for free from this site: https://ai.google.dev/gemini-api/docs/api-key?hl=en and paste it here: ")

# Variabes that can be changed

In [ ]:
prompt = "Write a short story on the minecraft server" #story prompt
additional_prompt = "cartoon style, 2d, cinematic, 8k" #additional prompt for pictures
hash_tags = "" # hashtags
la = "en" #language of video
division_on = 1 #Number between 1 and 60(control accelerate of video)
num_inference_steps = 25 #pictures quality
width = 1080 #width of video
height = 1920 #height of video

if division_on > 60 or division_on < 1:
    raise Exception("Variable division_on can receive value between 1 and 60 ONLY")

# Gemini-things

In [ ]:
gemini_api = API_KEY #gemini api-key

genai.configure(api_key=gemini_api)
model = genai.GenerativeModel(model_name="gemini-pro")

# Model download

In [ ]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
    print("CUDA is not available, falling back to CPU. This is bad")

model_id = "stabilityai/stable-diffusion-xl-base-1.0"

pipe = AutoPipelineForText2Image.from_pretrained(model_id, torch_dtype=torch.float16, variant="fp16")
pipe = pipe.to(device)

In [ ]:
translator = GoogleTranslator(source='auto', target='en')

# Functions

In [ ]:
class Generations:
    def __init__(self, prompt:str, additional_prompt:str, gemini_api:str, hash_tags:str, la:str, division_on:str, pipe, translator:GoogleTranslator, model:genai.GenerativeModel, num_inference_steps:int, width:int, height:int):
      try:
        self.prompt = prompt
        self.additional_prompt = additional_prompt
        self.gemini_api = gemini_api
        self.hash_tags = hash_tags
        self.la = la
        self.division_on = division_on
        self.pipe = pipe
        self.translator = translator
        self.model = model
        self.story = self.story_gen()
        self.story_dur = int(len(self.story.split(" ")) * (2/3))
        self.koef = (self.story_dur / self.division_on) + 0.3
        self.name = self.name_gen()
        self.num_inference_steps = num_inference_steps
        self.width = width
        self.height = height
      except Exception as ex:
        print(ex)

    def story_gen(self) -> str:
        responce = self.model.generate_content(self.prompt)
        print(f"Story:\n{responce.text}")
        return responce.text

    def chapters(self) ->list[list]:
        responce = self.model.generate_content(
        f"""
        Break this *text\n\n{self.story}\n\n into 10 (or more) logical parts (they should be approximately equal). Here is a strict template of the parts (no need to attribute anything additional):
        Only a brief description of the chapter (without the title of the chapter itself and without the chapter number)(so that an image can be generated from it)
        The duration on the next line (how many seconds will a person read the CONTENTS of the DESCRIBED CHAPTER (not short, but full of * text) at a reading speed of 85 WORDS per MINUTE (without a word of seconds) (integer))
        ...
        (description and duration on different lines (the duration line is exactly below the description line))(description on even lines (0, 2, 4, etc.)) (duration on odd lines (1, 3, 5, etc.)) (count the lines from scratch)(the line does not need to be numbered)(SEPARATE THE PARTS WITH INDENTS)

        Good generation (HOW TO GENERATE):
        Text:
        An epic battle between two powerful guilds was unfolding on the Minecraft survival server, hidden among endless forests and mountains: Knights of the Order and the Dragon Tribe.

        The battle raged for hours, the air was filled with screams and the clash of swords. Knights in shining armor came face to face with the warriors of the Dragon Tribe, whose scaly bodies shimmered in the moonlight.

        The leader of the Knights was the brave Sir Drexel, known for his indestructible will. At the head of the Dragon Tribe was a shrewd witch, Queen Elicia, who possessed ancient magic.

        When the battle reached its peak, Drexel and Elicia came together in a deadly duel. Sir Drexel waved his lightsaber, and Elicia wove spells that emitted lightning and flames.

        The battle was long and intense, but in the end, as the last warrior standing, Drexel defeated Elicia, thereby writing a new chapter in the history of the legendary battles of the Minecraft server.

        Table of contents and duration:
        Hidden Minecraft Server
        2

        Epic Guild Battle
        11

        Knights vs Dragon Tribe
        9

        Courage against Magic
        11

        Drexel and Elicia's duel
        11

        The relentless battle
        13

        The key point
        6

        Victory of the Knights
        8

        The Fall of Elicia
        8

        The Last Warrior
        9

        The second good generation (HOW TO GENERATE):
        Text:
        In a quiet corner of the virtual world, among lush forests and sparkling lakes, there was a Minecraft server named "Eden". His players were friendly and hardworking, built magnificent structures and created thriving communities.

        One day a new player named Alex joined. A fearless explorer, he traveled through the far lands of Eden, discovering unexplored territories and fighting dangerous creatures. But during one of these trips, he got lost.

        Night fell, and Alex realized that he was hopelessly lost. Despair gripped him as he wandered in the dark, looking for some sign of civilization. Suddenly, he saw a flickering light. He ran to him, hoping to find salvation.

        To his surprise, the light turned out to be a window into a modest house built by two other players, Sofia and Jake. Without hesitation, Alex knocked on the door, and it was joyfully opened to him. Sofia and Jake sheltered the lost traveler, offered him food and shelter.

        That night, sitting around the campfire, they told each other their stories. Alex's account of his adventures and tribulations made Sofia and Jake feel inspired, and their stories about life in
        Eden gave Alex the feeling of home he had longed for.

        At dawn, Alex thanked his new friends and said goodbye to them. Returning to his adventures, he was no longer alone and knew that if necessary, there would be those in Eden who would come to his aid. And the most valuable treasures in the wild world of Minecraft are friendship and human kindness.

        Table of contents and duration:
        1. Minecraft "Eden" server
        17

        2. Fearless traveler Alex
        16

        3. Alex is wandering in the dark
        20

        4. Twinkling light
        7

        5. Sophia and Jake's shelter
        17

        6. Stories around the campfire
        20

        7. Alex thanks his new friends
        26

        8. Upon returning to adventure
        9

        9. Friendship and kindness in Minecraft
        11

        10. The most valuable treasure
        12

        Bad generation (HOW NOT TO GENERATE)(The mistake is that the durations are too long):
        Text:
        An epic battle between two powerful guilds was unfolding on the Minecraft survival server, hidden among endless forests and mountains: Knights of the Order and the Dragon Tribe.

        The battle raged for hours, the air was filled with screams and the clash of swords. Knights in shining armor came face to face with the warriors of the Dragon Tribe, whose scaly bodies shimmered in the moonlight.

        The leader of the Knights was the brave Sir Drexel, known for his indestructible will. At the head of the Dragon Tribe was a shrewd witch, Queen Elicia, who possessed ancient magic.

        When the battle reached its peak, Drexel and Elicia came together in a deadly duel. Sir Drexel waved his lightsaber, and Elicia wove spells that emitted lightning and flames.

        The battle was long and intense, but in the end, as the last warrior standing, Drexel defeated Elicia, thereby writing a new chapter in the history of the legendary battles of the Minecraft server.

        Table of contents and duration:
        The battle on the Minecraft
        30 server

        Knights of the Order and the Dragon Tribe face off in a grand battle
        27

        Sir Drexel leads the Knights, Elicia leads the Dragon Tribe
        .

        Warriors fight in shining armor and scaly bodies
        27

        The battle rages for hours, the air is filled with screams and the clash of swords
        25

        Drexel and Elicia meet in a deadly duel
        24

        Drexel wields a lightsaber, Elicia weaves magic
        27

        The battle turns out to be intense and prolonged
        .

        Drexel defeats Elicia, becoming a legend
        33

        The result of the battle forever changes the history of the Minecraft server
        29

        The second bad generation (HOW NOT TO GENERATE)(The mistake is that the duration is fractional or approximate values):
        Text:
        An epic battle between two powerful guilds was unfolding on the Minecraft survival server, hidden among endless forests and mountains: Knights of the Order and the Dragon Tribe.

        The battle raged for hours, the air was filled with screams and the clash of swords. Knights in shining armor came face to face with the warriors of the Dragon Tribe, whose scaly bodies shimmered in the moonlight.

        The leader of the Knights was the brave Sir Drexel, known for his indestructible will. At the head of the Dragon Tribe was a shrewd witch, Queen Elicia, who possessed ancient magic.

        When the battle reached its peak, Drexel and Elicia came together in a deadly duel. Sir Drexel waved his lightsaber, and Elicia wove spells that emitted lightning and flames.

        The battle was long and intense, but in the end, as the last warrior standing, Drexel defeated Elicia, thereby writing a new chapter in the history of the legendary battles of the Minecraft server.

        Table of Contents and duration (HOW NOT TO GENERATE):
        The battle on the Minecraft server
        30.6

        Knights of the Order and the Dragon Tribe face off in a grand battle
        27.3

        Sir Drexel leads the Knights, Elicia leads the Dragon Tribe
        ~24.7

        Warriors fight in shining armor and scaly bodies
        27.1

        The battle rages for hours, the air is filled with screams and the clash of swords
        25.9

        Drexel and Elicia meet in a deadly duel
        24.2

        Drexel wields a lightsaber, Elicia weaves magic
        ~27.0

        The battle turns out to be intense and prolonged
        30.2

        Drexel defeats Elicia, becoming a legend
        33.1

        The result of the battle forever changes the history of the Minecraft server
        ~29.8

        The third bad example (HOW NOT TO GENERATE)(The mistake is that the duration and the table of contents are on the same line):
        Text:
        An epic battle between two powerful guilds was unfolding on the Minecraft survival server, hidden among endless forests and mountains: Knights of the Order and the Dragon Tribe.

        The battle raged for hours, the air was filled with screams and the clash of swords. Knights in shining armor came face to face with the warriors of the Dragon Tribe, whose scaly bodies shimmered in the moonlight.

        The leader of the Knights was the brave Sir Drexel, known for his indestructible will. At the head of the Dragon Tribe was a shrewd witch, Queen Elicia, who possessed ancient magic.

        When the battle reached its peak, Drexel and Elicia came together in a deadly duel. Sir Drexel waved his lightsaber, and Elicia wove spells that emitted lightning and flames.

        The battle was long and intense, but in the end, as the last warrior standing, Drexel defeated Elicia, thereby writing a new chapter in the history of the legendary battles of the Minecraft server.

        Table of contents and duration:
        Battle on the Minecraft server:30
        Knights of the Order and the Dragon Tribe face off in a grand battle:27
        Sir Drexel leads the Knights, Elicia leads the Dragon Tribe:24
        Warriors fight in shining armor and scaly bodies:27
        The battle rages for hours, the air is filled with screams and the clash of swords:25
        Drexel and Elicia meet in a deadly duel:24
        Drexel wields a lightsaber, Elicia weaves magic:27
        The battle turns out to be intense and prolonged:30
        Drexel defeats Elicia, becoming a legend:33
        The result of the battle forever changes the history of the Minecraft server:29

        The third bad generation (HOW NOT TO GENERATE)(The mistake is that the durations and table of contents do not converge)
        Text:
        On the Eldorado server, the persistent builder Oliver was building a grandiose cathedral. It was the fruit of his many months of work, and he was proud of every hand-carved block.

        One day, when Oliver was almost done, a griefer named Slime broke into the server. Slime began to destroy everything in his path, igniting TNT and sweeping away buildings. Oliver watched in horror as his cathedral turned into a pile of rubble.

        In desperation, Oliver called the administrator for help. The administrator reacted quickly and banished Slime from the server. But Oliver's cathedral was destroyed.

        Oliver was devastated, but soon his resolve returned. With the help of other players on the server, he began to rebuild the cathedral, introducing additional security measures this time to protect it from vandals.

        A few months later, Oliver's Cathedral was finished, even more magnificent than before. Players from all over the server came to admire his creation, and Oliver became the legend of El Dorado.

        Table of contents and duration:
        - A persistent builder is building a cathedral on server
        5

        - Griefer destroys Cathedral
        6

        - The administrator expels the griefer
        8

        - The builder restores the cathedral with protection from vandals
        10

        - Completion of the magnificent cathedral
        7

        - The builder becomes a legend of server
        9

        - Oliver is proud of the hand-carved blocks
        12

        - Oliver is in despair after the destruction of the cathedral
        8

        - Oliver restores the cathedral with the help of the players
        10

        The fourth bad generation (HOW NOT TO GENERATE)(The error is that an additional line is allocated to number the part)
        Text:
        In the thousand-year history, the Great Kazakh Empire has faced countless threats. But nothing could compare to the horror emanating from the Evil Empire.

        Led by the shadow Lord Antagonist, this empire sowed cruelty and destruction in its path. Their armies, filled with darkness and demonic forces, sought to enslave everything that came their way.

        Kazakh soldiers under the leadership of the wise Khan Alpamys stood up to defend their empire. Horsemen and archers, they fought with unwavering bravery.

        The battle raged on the endless steppes, where every inch of the ground was soaked in blood and tears. Swords flashed, arrows flew with a whistling sound, and hordes of evil crushed everything in their path.

        But the Kazakhs did not give up. They drew strength from their Nomadic ancestors, their spirit was indestructible. They fought for their homes, their land and their freedom.

        The battle went on for centuries, every inch of the steppes turned into a battlefield. But soon hope penetrated the ranks of the Kazakhs. An ancient prophecy predicted the return of a legendary warrior who would lead them to victory.

        And so he came. From the depths of the past, Madi emerged, a legendary warrior whose strength surpassed all who had come before him. His sword blazed with heavenly fire, and his army fought with unwavering courage.

        Together, Madi and Alpamys led the last onslaught of the Kazakhs. They broke through the hordes of evil, fighting shoulder to shoulder, their swords wreaking havoc among the enemies.

        Finally, after centuries of bloodshed, the Evil Empire fell. Antagon was defeated, and his dark armies were thrown into the abyss.

        The great Kazakh Empire triumphed. They stood up to the greatest threat their people have ever faced. Their courage and unwavering spirit have become a legend that has been passed down from generation to generation.

        Table of contents and duration:
        0.
        The battle of the Kazakh Empire with the Evil Empire
        67

        2.
        Kazakhstani warriors defend their empire
        58

        4.
        The appearance of Madi, the legendary warrior
        47

        6.
        The last onslaught of the Kazakhs
        52

        8.
        The victory of the Kazakh Empire
        46

        10.
        Courage and unwavering spirit of the Kazakhs
        51
        """)

        chapters = responce.text
        print(f"Chapters:\n{chapters}")
        buf = StringIO(chapters)
        name = []
        duration = []
        count = 0
        try:
            while True:
                out = buf.readline()
                if out == "\n":
                    continue
                if count % 2 == 0:
                    name.append(out)
                else:
                    duration.append(int(out))
                count += 1
                if name[len(name) - 1] == '':
                    name.remove('')
                    break
            print([name, duration])
            self.bot_dur = sum(duration)
            print(f"Bot's story duration: {self.bot_dur}")
            return [name, duration]
        except Exception as ex:
            print(ex)
            return [[],[]]

    def name_gen(self):
        responce = self.model.generate_content(
    f"Title this text {self.story}. Generate only the name, nothing additional (the name should not contain quotation marks (These - \"))(Translate this title on {self.la})"
        )
        return f"{responce.text} {self.hash_tags}"

    def check(self, duration:int)->bool:
        if duration - self.story_dur > 15 or duration - self.story_dur < -7:
            return False
        else:
            return True

    def images_gen(self, images:list):
        count = 0

        for image in images:
            image_gen = self.pipe(f"{self.translator.translate(image+'('+self.additional_prompt+')')}", num_inference_steps=self.num_inference_steps).images[0]
            image_gen.save(f"{count}.png")
            count += 1

    def audio_gen(self) -> None:
        speak = gTTS(text=self.story, lang=self.la, slow=False)
        speak.save("audio.mp3")
        print("Audio is saved")

    def accelerate(self):
        video_path = f"{self.name}Slow.mp4"
        output_video_path = f"{self.name}.mp4"

        video = VideoFileClip(video_path)
        audio_path = "extracted_audio.wav"
        video.audio.write_audiofile(audio_path)

        song, fs = librosa.load(audio_path, sr=None)
        songfaster = librosa.effects.time_stretch(song, rate=self.koef)

        fade_duration = self.bot_dur - self.story_dur
        if fade_duration > 0:
            fade_out_samples = int(fs * fade_duration)
            if len(songfaster) >= fade_out_samples:
                songfaster[-fade_out_samples:] = 0

        def highpass_filter(data, cutoff=100, fs=44100, order=6):
            b, a = butter(order, cutoff / (fs / 2), btype='highpass')
            return filtfilt(b, a, data)

        songfaster = highpass_filter(songfaster, cutoff=100, fs=fs)

        output_audio_path = "audioAc.wav"
        wavfile.write(output_audio_path, fs, (songfaster * 32767).astype(np.int16))

        new_audio = AudioFileClip(output_audio_path)
        new_video = video.fx(vfx.speedx, self.koef)

        new_audio = new_audio.subclip(0, new_video.duration)
        final_video = new_video.set_audio(new_audio)

        final_video.write_videofile(output_video_path, codec="libx264")

    def video_gen(self, timecodes:list) -> None:
        count = 0
        clips = []
        for timecode in timecodes:
            clips.append(ImageClip(f"{count}.png").set_duration(timecode))
            count += 1
        concat_clip = concatenate_videoclips(clips, method="compose")
        concat_clip.audio = AudioFileClip("audio.mp3")
        resized_clip = concat_clip.resize((self.width, self.height))

        resized_clip.write_videofile(f"{self.name}Slow.mp4", fps=24)
        resized_clip.close()
        self.accelerate()

# Generations object

In [ ]:
Generation = Generations(prompt, additional_prompt, gemini_api, hash_tags, la, division_on, pipe, translator, model, num_inference_steps, width, height)

# Table of contents of parts of the text

In [ ]:
while True:
  try:
    n = Generation.chapters()
    print(f"Story durtion : {Generation.story_dur}")
    if Generation.check(sum(n[1])) and n != [[]]:
        print("Succes!!!")
        break
  except Exception as ex:
    print(ex)
    break
  sleep(5)

# Image and audio generation

In [ ]:
try:
  image_thread = threading.Thread(target=Generation.images_gen, args=(n[0],))
  image_thread.start()

  Generation.audio_gen()

  image_thread.join()
except Exception as ex:
  print(ex)

# Video generation

In [ ]:
try:
  Generation.video_gen(n[1])

  print("Succesfully created new video")
except Exception as ex:
  print(ex)